# Proyecto Final — Inferencia sobre Datos Nuevos

En este notebook aplicamos los modelos entrenados sobre datos nuevos para evaluar su capacidad de generalización. El flujo garantiza coherencia con el entrenamiento mediante la reutilización de pipelines y metadatos, permitiendo inferencia reproducible sobre cualquier dataset con la estructura esperada.


## Configuración y dependencias

Definimos las rutas a los artefactos entrenados y al dataset de inferencia. Para aplicar el modelo sobre datos nuevos, modificamos `DATA_FILE` apuntando al archivo correspondiente manteniendo la misma estructura de columnas del entrenamiento.


In [1]:
from typing import Literal, Tuple
from pathlib import Path
import numpy as np
import pandas as pd
import joblib
import warnings
warnings.filterwarnings('ignore')

# Configuración de la tarea
TAREA: Literal["clasificacion", "regresion"] = "clasificacion"

# Rutas - Modificar DATA_FILE para aplicar sobre datos nuevos
DATA_FILE = Path("data/interim/supermercado_limpio.csv")

MODEL_FILE = Path("models/pipeline_clasificacion_sin_leakage.pkl") if TAREA == "clasificacion" else Path("models/pipeline_regresion_sin_leakage.pkl")

METADATA_FILE = Path("models/pipeline_metadata.pkl")
TARGET_COL = "respuesta" if TAREA == "clasificacion" else "gasto_total"

## Carga del dataset de inferencia

Cargamos el dataset sobre el cual aplicaremos el modelo. Este debe tener la misma estructura de columnas que el dataset de entrenamiento, aunque pueden diferir en el número de observaciones.


In [2]:
assert DATA_FILE.exists(), f"Dataset no encontrado: {DATA_FILE}"

df_raw = pd.read_csv(DATA_FILE)

print(f"Dataset cargado: {df_raw.shape[0]} observaciones, {df_raw.shape[1]} variables")

print(f"Variable objetivo disponible: {TARGET_COL in df_raw.columns}")
df_raw.head()


Dataset cargado: 1982 observaciones, 48 variables
Variable objetivo disponible: True


,educacion,estado_civil,ingresos,hijos_casa,recencia,gasto_vinos,gasto_frutas,gasto_carnes,gasto_pescado,gasto_dulces,...,tasa_compra_online,tasa_compra_oferta,ticket_promedio,tamano_hogar,tiene_dependientes,hogar_unipersonal,ratio_compras_online,tiene_pareja,educacion_x_estado,anio_alta
0,3,Casado,53359.0,2,4,173,4,30,3,6,...,0.429,0.286,18.36,3,1,0,0.428571,1,3,2013
1,3,Soltero,21474.0,1,0,6,16,24,11,0,...,0.500,0.250,11.38,2,1,0,0.500000,0,0,2014
2,3,Divorciado,41411.0,0,11,37,32,38,11,3,...,0.375,0.125,17.38,1,0,1,0.375000,0,0,2013
3,5,Union_Libre,64504.0,3,81,986,36,168,16,0,...,0.560,0.280,52.56,4,1,0,0.560000,1,5,2013
4,3,Casado,65169.0,0,23,1074,0,69,0,0,...,0.500,0.036,42.46,1,0,1,0.500000,1,3,2014


## Carga de artefactos de entrenamiento

Cargamos el pipeline entrenado y los metadatos que contienen información sobre las features esperadas, transformaciones aplicadas y configuración del modelo.


In [3]:
assert METADATA_FILE.exists(), f"Metadatos no encontrados: {METADATA_FILE}"
assert MODEL_FILE.exists(), f"Modelo no encontrado: {MODEL_FILE}"

metadata = joblib.load(METADATA_FILE)
model = joblib.load(MODEL_FILE)


print(f"Pipeline cargado: {type(model).__name__}")

if TAREA == "clasificacion":    print(f"Umbral de clasificación: {metadata.get('clf_threshold', 0.5):.4f}")

Pipeline cargado: Pipeline
Umbral de clasificación: 0.2069


## Funciones auxiliares

Definimos funciones para validar y preparar las features del dataset de inferencia, garantizando alineación exacta con el espacio de features utilizado durante el entrenamiento.


In [4]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)


def validar_columnas(work: pd.DataFrame, expected: list[str], contexto: str, allow_extras: bool = False) -> pd.DataFrame:
    # Verifica alineación de columnas; opcionalmente descarta extras
    faltantes = [c for c in expected if c not in work.columns]
    extras = [c for c in work.columns if c not in expected]
    if faltantes:
        msg = (
            f"Desalineación de columnas en {contexto}. "
            f"Faltantes: {faltantes or 'ninguna'} | Extras: {extras or 'ninguna'}"
        )
        raise ValueError(msg)
    if extras and not allow_extras:
        msg = (
            f"Desalineación de columnas en {contexto}. "
            f"Faltantes: {faltantes or 'ninguna'} | Extras: {extras or 'ninguna'}"
        )
        raise ValueError(msg)
    if extras:
        print(f"Descartamos columnas extra en {contexto}: {extras}")
        work = work.drop(columns=extras)
    return work[expected]


def preparar_features(df: pd.DataFrame) -> Tuple[pd.DataFrame, pd.Series | None]:
    work = df.drop(columns=[TARGET_COL], errors="ignore").copy()
    for col in work.columns:
        if str(work[col].dtype) == "Int64":
            work[col] = work[col].astype("int64")
    target = df[TARGET_COL] if TARGET_COL in df.columns else None
    return work, target


def preparar_regresion(df: pd.DataFrame, meta: dict) -> Tuple[pd.DataFrame, pd.Series | None]:
    work = df.drop(columns=[TARGET_COL], errors="ignore").copy()
    expected = meta.get("reg_raw_feature_names") or meta["raw_feature_names"]
    work = validar_columnas(work, expected, "regresion (features crudas)", allow_extras=True)
    for col in work.columns:
        if str(work[col].dtype) == "Int64":
            work[col] = work[col].astype("int64")
    target = df[TARGET_COL] if TARGET_COL in df.columns else None
    return work, target


## Preparación del dataset para inferencia

Extraemos las features y la variable objetivo (si está disponible), validando que las columnas coincidan con las esperadas por el modelo.

In [5]:
# Determinar columnas esperadas
preproc = getattr(model, "named_steps", {}).get("preprocess") if hasattr(model, "named_steps") else None
preproc_cols = []
if preproc is not None and hasattr(preproc, "transformers_"):
    for _, _, cols in preproc.transformers_:
        if cols == "drop" or cols is None:
            continue
        if isinstance(cols, (list, tuple)):
            preproc_cols.extend(list(cols))
        else:
            try:
                preproc_cols.extend(list(cols))
            except TypeError:
                pass

# Preparar features según tarea
if TAREA == "clasificacion":
    X_inf, y_inf = preparar_features(df_raw)
    expected_cols = metadata.get("raw_feature_names", [])
    if preproc_cols:
        expected_cols = list(set(expected_cols) | set(preproc_cols))
    X_inf = validar_columnas(X_inf, expected_cols, "clasificacion", allow_extras=True)
elif TAREA == "regresion":
    X_inf, y_inf = preparar_regresion(df_raw, metadata)
    expected_cols = metadata.get("reg_raw_feature_names") or metadata.get("raw_feature_names", [])
    if preproc_cols:
        expected_cols = list(set(expected_cols) | set(preproc_cols))
    X_inf = validar_columnas(X_inf, expected_cols, "regresion", allow_extras=True)
else:
    raise ValueError(f"Tarea no reconocida: {TAREA}")

print(f"Features preparadas: {X_inf.shape[0]} observaciones, {X_inf.shape[1]} variables")
print(f"Target disponible: {y_inf is not None}")

# Resumen
resumen_dimensiones = pd.DataFrame({
    "observaciones": [X_inf.shape[0]],
    "features": [X_inf.shape[1]],
    "target_disponible": [y_inf is not None]
})
resumen_dimensiones

Features preparadas: 1982 observaciones, 47 variables
Target disponible: True


,observaciones,features,target_disponible
0,1982,47,True


## Generación de predicciones

Aplicamos el pipeline entrenado sobre el dataset de inferencia.

In [6]:
if TAREA == "clasificacion":
    y_pred_proba = model.predict_proba(X_inf)[:, 1]
    clf_threshold = float(metadata.get("clf_threshold", 0.5))
    y_pred = (y_pred_proba >= clf_threshold).astype(int)
    
    print(f"Umbral aplicado: {clf_threshold:.4f}")
    distrib_pred = pd.Series(y_pred).value_counts().sort_index().to_frame("Frecuencia")
    distrib_pred["Porcentaje"] = (distrib_pred["Frecuencia"] / len(y_pred) * 100).round(2)
    distrib_pred.index = ["No Responde", "Responde"]
    distrib_pred
else:
    y_pred = model.predict(X_inf)
    
    print("Estadísticas de las predicciones:")
    resumen_pred = pd.DataFrame({
        "y_pred": pd.Series(y_pred).describe(),
    })
    resumen_pred

Umbral aplicado: 0.2069


## Evaluación de métricas

Si el dataset incluye la variable objetivo, calculamos las métricas de rendimiento para evaluar la capacidad de generalización del modelo.

In [7]:
if TAREA == "clasificacion":
    y_pred_proba = model.predict_proba(X_inf)[:, 1]
    clf_threshold = float(metadata.get("clf_threshold", 0.5))
    y_pred = (y_pred_proba >= clf_threshold).astype(int)
elif TAREA == "regresion":
    expected_cols = metadata.get("reg_raw_feature_names") or metadata.get("raw_feature_names", [])
    X_inf = validar_columnas(X_inf, expected_cols, "regresion (features crudas)", allow_extras=True)

In [8]:
if y_inf is not None:
    if TAREA == "clasificacion":
        metrics_clf = pd.DataFrame({
            "AUC": [roc_auc_score(y_inf, y_pred_proba)],
            "Accuracy": [accuracy_score(y_inf, y_pred)],
            "Precision": [precision_score(y_inf, y_pred)],
            "Recall": [recall_score(y_inf, y_pred)],
            "F1": [f1_score(y_inf, y_pred)],
        }, index=["inferencia"])
        
        matriz_confusion = pd.DataFrame(
            confusion_matrix(y_inf, y_pred),
            index=["Real 0", "Real 1"],
            columns=["Pred 0", "Pred 1"],
        )
        display(metrics_clf)
        matriz_confusion
    else:
        baseline_mae = (y_inf - y_inf.mean()).abs().mean()
        baseline_rmse = ((y_inf - y_inf.mean()) ** 2).mean() ** 0.5
        mae = mean_absolute_error(y_inf, y_pred)
        rmse = mean_squared_error(y_inf, y_pred) ** 0.5
        r2 = r2_score(y_inf, y_pred)
        
        metrics_reg = pd.DataFrame({
            "MAE": [mae],
            "RMSE": [rmse],
            "R2": [r2],
            "baseline_MAE": [baseline_mae],
            "baseline_RMSE": [baseline_rmse],
        }, index=["inferencia"])
        
        display(metrics_reg)
        distrib_pred = pd.DataFrame({
            "y_real": y_inf.describe(),
            "y_pred": pd.Series(y_pred).describe(),
        })
        distrib_pred

,AUC,Accuracy,Precision,Recall,F1
inferencia,0.981213,0.935923,0.715493,0.907143,0.8


## Síntesis

Cargamos los pipelines entrenados y aplicamos las predicciones sobre el dataset de inferencia. Si está disponible la variable objetivo, calculamos las métricas correspondientes para evaluar el rendimiento del modelo.

## Conclusiones

El notebook carga el pipeline entrenado y genera predicciones sobre el dataset de inferencia. Las métricas obtenidas permiten evaluar el desempeño del modelo sobre datos nuevos.